In [1]:
# Import packages
import nltk
from nltk.corpus import stopwords
from pprint import pprint
import yaml
import sys
import os
import re
import requests
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import stanza
from nltk.probability import FreqDist
import re
import numpy as np
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
# NLTK Stop words
# from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
enron_emails_df=pd.read_csv('../data/main_data.csv')
enron_emails_df.describe()

,Time,Total_Sentence_Word_Count
count,480062.000000,480062.000000
mean,1999.504372,250.498367
std,47.500908,764.799161
min,1.000000,10.000000
25%,2000.000000,48.000000
50%,2001.000000,114.000000
75%,2001.000000,247.000000
max,2044.000000,60047.000000


In [5]:
enron_emails_df["Content"].head()

0    Traveling to have a business meeting takes the...
1    Randy,\n\n Can you send me a schedule of the s...
2    Please cc the following distribution list with...
3    1. login:  pallen pw: ke9davis\n\n I don't thi...
4    ---------------------- Forwarded by Phillip K ...
Name: Content, dtype: object

In [6]:
# Convert to list
data = enron_emails_df['Content'].values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21332\1450432846.py:5: SyntaxWarning: invalid escape sequence '\S'
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21332\1450432846.py:8: SyntaxWarning: invalid escape sequence '\s'
  data = [re.sub('\s+', ' ', sent) for sent in data]


['Traveling to have a business meeting takes the fun out of the trip. '
 'Especially if you have to prepare a presentation. I would suggest holding '
 'the business plan meetings here then take a trip without any formal business '
 'meetings. I would even try and get some honest opinions on whether a trip is '
 'even desired or necessary. As far as the business meetings, I think it would '
 'be more productive to try and stimulate discussions across the different '
 'groups about what is working and what is not. Too often the presenter speaks '
 'and the others are quiet just waiting for their turn. The meetings might be '
 'better if held in a round table discussion format. My suggestion for where '
 'to go is Austin. Play golf and rent a ski boat and jet skis. Flying '
 'somewhere takes too much time.']


In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['traveling', 'to', 'have', 'business', 'meeting', 'takes', 'the', 'fun', 'out', 'of', 'the', 'trip', 'especially', 'if', 'you', 'have', 'to', 'prepare', 'presentation', 'would', 'suggest', 'holding', 'the', 'business', 'plan', 'meetings', 'here', 'then', 'take', 'trip', 'without', 'any', 'formal', 'business', 'meetings', 'would', 'even', 'try', 'and', 'get', 'some', 'honest', 'opinions', 'on', 'whether', 'trip', 'is', 'even', 'desired', 'or', 'necessary', 'as', 'far', 'as', 'the', 'business', 'meetings', 'think', 'it', 'would', 'be', 'more', 'productive', 'to', 'try', 'and', 'stimulate', 'discussions', 'across', 'the', 'different', 'groups', 'about', 'what', 'is', 'working', 'and', 'what', 'is', 'not', 'too', 'often', 'the', 'presenter', 'speaks', 'and', 'the', 'others', 'are', 'quiet', 'just', 'waiting', 'for', 'their', 'turn', 'the', 'meetings', 'might', 'be', 'better', 'if', 'held', 'in', 'round', 'table', 'discussion', 'format', 'my', 'suggestion', 'for', 'where', 'to', 'go', 'is

In [8]:
'''
Bigrams are two words frequently occurring together in the document.
Trigrams are 3 words frequently occurring.

Some examples in our example are: ‘front_bumper’, ‘oil_leak’, ‘maryland_college_park’ etc.

Gensim’s Phrases model can build and implement the bigrams, trigrams, 
quadgrams and more.

The two important arguments to Phrases are min_count and threshold.
The higher the values of these param, the harder it is for words to be combined to bigrams.
'''
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['traveling', 'to', 'have', 'business', 'meeting', 'takes', 'the', 'fun', 'out', 'of', 'the', 'trip', 'especially', 'if', 'you', 'have', 'to', 'prepare', 'presentation', 'would', 'suggest', 'holding', 'the', 'business', 'plan', 'meetings', 'here', 'then', 'take', 'trip', 'without', 'any', 'formal', 'business', 'meetings', 'would', 'even', 'try', 'and', 'get', 'some', 'honest', 'opinions', 'on', 'whether', 'trip', 'is', 'even', 'desired', 'or', 'necessary', 'as', 'far', 'as', 'the', 'business', 'meetings', 'think', 'it', 'would', 'be', 'more', 'productive', 'to', 'try', 'and', 'stimulate', 'discussions', 'across', 'the', 'different', 'groups', 'about', 'what', 'is', 'working', 'and', 'what', 'is', 'not', 'too', 'often', 'the', 'presenter', 'speaks', 'and', 'the', 'others', 'are', 'quiet', 'just', 'waiting', 'for', 'their', 'turn', 'the', 'meetings', 'might', 'be', 'better', 'if', 'held', 'in', 'round', 'table', 'discussion', 'format', 'my', 'suggestion', 'for', 'where', 'to', 'go', 'is'

In [9]:
'''
The bigrams model is ready. Let’s define the functions to remove the stopwords, 
make bigrams and lemmatization and call them sequentially.
'''
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
'''
Let’s call the functions in order.
'''
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

'''
If it reports error "Can't find model 'en'. It doesn't seem to be a shortcut link, 
a Python package or a valid path to a data directory", you need to do following in command line:

> python -m spacy download en

It works only if you have administrator rights as the installation folder needs that. Otherwise
you will receive error "Download successful but linking failed. Creating a shortcut link for 'en' 
didn't work (maybe you don't have admin permissions?), but you can still load the model via its 
full package name: nlp = spacy.load('en_core_web_sm')". It may not work even you do as suggested.
In such case, you need to open Anadaconda's cmd prompt as an Administrator:

1. Right clik on the Anaconda Promt Icon
2. Go to "More" and click on "Open as Administrator"
3. Then type: python -m spacy download en
'''

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['travel', 'business', 'meeting', 'take', 'fun', 'trip', 'especially', 'prepare', 'presentation', 'suggest', 'hold', 'business', 'plan', 'meeting', 'take', 'trip', 'formal', 'business', 'meeting', 'even', 'try', 'get', 'honest', 'opinion', 'trip', 'even', 'desire', 'necessary', 'far', 'business', 'meeting', 'think', 'productive', 'try', 'stimulate', 'discussion', 'different', 'group', 'work', 'often', 'presenter', 'speak', 'other', 'quiet', 'waiting', 'turn', 'meeting', 'well', 'hold', 'round', 'table', 'discussion', 'format', 'suggestion', 'go', 'play', 'golf', 'rent', 'ski', 'boat', 'jet_ski', 'fly', 'somewhere', 'take', 'much', 'time']]


In [11]:
'''
The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus.
Let’s create them.
'''
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 4), (2, 1), (3, 1), (4, 2), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 5), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 3), (42, 1), (43, 1), (44, 1), (45, 3), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1)]]


In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=123,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.043*"datum" + 0.038*"information" + 0.026*"report" + 0.024*"email" + '
  '0.023*"system" + 0.018*"contact" + 0.018*"unknown_database" + 0.018*"order" '
  '+ 0.016*"access" + 0.015*"error"'),
 (1,
  '0.054*"table" + 0.049*"font" + 0.046*"width" + 0.038*"border" + '
  '0.038*"size" + 0.037*"class" + 0.036*"align" + 0.033*"test" + '
  '0.032*"width_height" + 0.021*"participate"'),
 (2,
  '0.043*"agreement" + 0.032*"credit" + 0.027*"purchase" + 0.026*"rate" + '
  '0.023*"notice" + 0.023*"charge" + 0.022*"pay" + 0.020*"payment" + '
  '0.019*"sale" + 0.019*"bill"'),
 (3,
  '0.207*"mail" + 0.067*"intend" + 0.064*"copy" + 0.063*"receive" + '
  '0.046*"contract" + 0.035*"policy" + 0.028*"distribution" + 0.026*"offer" + '
  '0.024*"ahead" + 0.022*"otherwise"'),
 (4,
  '0.093*"send" + 0.080*"message" + 0.064*"original" + 0.058*"thank" + '
  '0.044*"know" + 0.035*"need" + 0.034*"let" + 0.018*"see" + 0.018*"regard" + '
  '0.015*"like"'),
 (5,
  '0.165*"com" + 0.088*"http" + 0.045*"click" 

In [14]:
'''
Model perplexity and topic coherence provide a convenient measure to judge 
how good a given topic model is. In my experience, topic coherence score, 
in particular, has been more helpful.
'''
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score (how consistent the topic is)
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

'''
Perplexity:  -8.86067503009

Coherence Score:  0.532947587081


There you have a coherence score of 0.53.
'''


Perplexity:  -11.874786318979345

Coherence Score:  0.47056894727902804


'\nPerplexity:  -8.86067503009\n\nCoherence Score:  0.532947587081\n\n\nThere you have a coherence score of 0.53.\n'

In [15]:
'''
Now that the LDA model is built, the next step is to examine the produced 
topics and the associated keywords. There is no better tool than pyLDAvis package’s 
interactive chart and is designed to work well with jupyter notebooks.
'''
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

In [16]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.235658 -0.091642       1        1  15.482905
14    -0.191540  0.119116       2        1  11.462410
7     -0.186346  0.001922       3        1  10.847555
4     -0.195869 -0.163677       4        1   9.430508
10    -0.101947  0.073557       5        1   8.810949
0     -0.091580  0.020708       6        1   8.396278
2     -0.032781  0.041477       7        1   6.296397
5      0.102290  0.006497       8        1   5.082201
9      0.059612  0.236030       9        1   4.886303
8      0.079909 -0.086252      10        1   4.223639
13     0.116406  0.256580      11        1   4.205493
6      0.070735 -0.173154      12        1   3.526616
11     0.198753 -0.129746      13        1   2.592782
1      0.188567 -0.163483      14        1   2.378706
3      0.219449  0.052068      15        1   2.377258, topic_info=          Term           Freq          Total Category  logprob  loglift
1007       com  436454.000000  436454.000000  Default  30.0000  30.0000
165      enron  467465.000000  467465.000000  Default  29.0000  29.0000
61        send  453597.000000  453597.000000  Default  28.0000  28.0000
133      power  441248.000000  441248.000000  Default  27.0000  27.0000
431       mail  255336.000000  255336.000000  Default  26.0000  26.0000
...        ...            ...            ...      ...      ...      ...
533   contract   57064.443891  125174.767820  Topic15  -3.0738   2.9537
980      basis   23657.599307   49170.841478  Topic15  -3.9543   3.0076
477      offer   32274.503646  128202.395245  Topic15  -3.6437   2.3599
624      party   22816.702648   69391.691169  Topic15  -3.9905   2.6269
485     review   25127.657778  112399.999379  Topic15  -3.8940   2.2411

[716 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2495     13  0.999931  acceptance
374       6  0.999977      access
1525      2  0.999949      accord
77        6  0.213302     account
77        7  0.786669     account
...     ...       ...         ...
1720      5  0.999948       world
1482      8  0.999973         www
161       1  0.409723        year
161       2  0.357768        year
161       5  0.232502        year

[985 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 15, 8, 5, 11, 1, 3, 6, 10, 9, 14, 7, 12, 2, 4])